# Query execution

Our example query on the Reddit dataset is:

In [7]:
import polars as pl

q1 = (
    pl.scan_csv("../../datasets/reddit.csv")
    .with_columns(pl.col("name").str.to_uppercase())
    .filter(pl.col("comment_karma") > 0)
)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [../../datasets/reddit.csv]
PROJECT */6 COLUMNS
ESTIMATED ROWS: 88


If we were to run the code above on the Reddit CSV the query would not be evaluated. Instead Polars takes each line of code, adds it to the internal query graph and optimizes the query graph.

When we execute the code Polars executes the optimized query graph by default.

## Execution on the full dataset

We can execute our query on the full dataset by calling the .collect method on the query.

In [14]:
q4 = (
    pl.scan_csv(f"../../datasets/reddit.csv")
    .with_columns(pl.col("name").str.to_uppercase())
    .filter(pl.col("comment_karma") > 0)
    .collect()
)

q4

id,name,created_utc,updated_on,comment_karma,link_karma
i64,str,i64,i64,i64,i64
6,"""TAOJIANLONG_JASONBROKEN""",1397113510,1536527864,4,0
17,"""SSAIG_JASONBROKEN""",1397113544,1536527864,1,0
19,"""FDBVFDSSDGFDS_JASONBROKEN""",1397113552,1536527864,3,0
37,"""IHATEWHOWEARE_JASONBROKEN""",1397113636,1536527864,61,0
77714,"""HOCKEYSCHTICK""",1137474000,1536497404,11104,451
…,…,…,…,…,…
77762,"""COOLIN86""",1138303196,1536528275,40,7
77763,"""LUNCHY""",1137599510,1536528275,65,0
77765,"""COMPOSTELLAS""",1137474000,1536528276,6,0


Above we see that from the 10 million rows there are 14,029 rows that match our predicate.

With the default collect method Polars processes all of your data as one batch. This means that all the data has to fit into your available memory at the point of peak memory usage in your query.

## Execution on larger-than-memory data
If your data requires more memory than you have available Polars may be able to process the data in batches using *streaming mode*. To use streaming mode you simply pass the `engine="streaming"` argument to `collect`

In [16]:
q5 = (
    pl.scan_csv(f"../../datasets/reddit.csv")
    .with_columns(pl.col("name").str.to_uppercase())
    .filter(pl.col("comment_karma") > 0)
    .collect(engine='streaming')
)

## Execution on a partial dataset

While you're writing, optimizing or checking your query on a large dataset, querying all available data may lead to a slow development process.

Instead, you can scan a subset of your partitions or use `.head`/`.collect` at the beginning and end of your query, respectively. Keep in mind that the results of aggregations and filters on subsets of your data may not be representative of the result you would get on the full data.

In [18]:
q9 = (
    pl.scan_csv(f"../../datasets/reddit.csv")
    .head(10)
    .with_columns(pl.col("name").str.to_uppercase())
    .filter(pl.col("comment_karma") > 0)
    .collect()
)

## Diverging queries
It is very common that a query diverges at one point. In these cases it is recommended to use collect_all as they will ensure that diverging queries execute only once.

In [ ]:
# Some expensive LazyFrame

lf: LazyFrame

lf_1 = lf.select(pl.all().sum())

lf_2 = lf.some_other_computation()

pl.collect_all([lf_1, lf_2]) # this will execute lf only once!